## Описание проекта
Автор: Зайцев Виталий  
Данные взяты с https://datahelpdesk.worldbank.org/knowledgebase

### Цель проекта

Разработать автоматизированный pipeline для сбора, обработки и анализа данных Всемирного банка (получение ключевых показателей через API). Визуализировать ключевые метрики и проверить гипотезу о связи экономического роста и доступности благ для населения. Результатом будут интерактивный дашборд и отчет (Jupyter), который можно будет легко воспроизвести при наличии скрипта pipeline.

### Стек технологий
- Язык программирования: Python 3.12.7
- Основные библиотеки: requests, pandas, sqlalchemy, seaborn, plotly
- Для дашборда: datalens
- Для документации / отчета: Jupyter Notebook
- Система контроля версий: GitHub

### План работы
1. Pipeline – извлечение данных с сайта Всемирного банка / трансформация / загрузка в базу данных (ETL)
2. Анализ и визуализация (Jupyter)
3. Создание аналитической панели (DataLens)

---
## Что сделано
### 1. Pipeline – извлечение данных с сайта Всемирного банка / трансформация / загрузка в базу данных (ETL)

Данные извлекались по API [Всемирного банка](https://datahelpdesk.worldbank.org/knowledgebase/topics/125589).

После трансформации в скрипте данные загружались в базу данных postresql на Supabase.

Созданы 3 скрипта: 
- `load_countries.py` - извлекает справочник по всем странам, удаляет строки с агрегацией, перобразовывает типы данных, загружает в базу данных таблицу `worldbank_countries`.
- `load_indicators.py` - извлекает справочник всех показателей (индикаторов), удаляет дубликаты, загружает в базу данных таблицу `worldbank_indicators`.
- `load_indicators_values.py` - извлекает значения показателей для всех стран по выбранным показателям, преобразует данные, загружает в базу данных таблицу `worldbank_values`.

---

### 2. Анализ и визуализация

В Supabase таблица `worldbank_values` была разбита на 2 через представления (view):
- `all_countries` - отобраны только страны
  
```sql
  CREATE OR REPLACE VIEW all_countries AS 
    SELECT wv.country, wv.indicator, wv.year, wv.value, wv.country_id, wc.income_level_name
    FROM worldbank_values wv 
    JOIN worldbank_countries wc USING(country_id);
```

- `aggregate_countries` - отобраны только регионы и общие названия территорий. Сюда же вошла категоризация по уровню дохода (incom).
    
    ```sql
    CREATE VIEW aggregate_countries AS 
        SELECT country, indicator, year, value
        FROM worldbank_values
        WHERE country NOT IN (
              SELECT country_name FROM worldbank_countries
        );
    ```
Эти таблицы использовались для анализа данных в тетрадке `wb_analysis.ipynb`.

В ходе анализа была сделана кластеризация стран методом K-Means, страны были разбиты на 4 кластера по трем параметрам - ВВП на душу населения, Доступ к электричеству и Городское население. Результат такой сегментации был сохранен в базу данных в таблицу `all_country_with_segmentation`.

### Создание аналитической панели (дашборд)

#### Подготовка витрины данных

В базу данных была добавлена таблица с полигонами стран (данные взяты с сайта Datalens).

Для дашборда в базе данных была создана витрина данных (материализованное представление) `countries_segments_poligons`:

```sql
CREATE materialized VIEW countries_segments_poligons AS 
SELECT s.country, s.year, 
      max(case when s.indicator = 'Access to electricity (% of population)'
              then s.value end) as access_to_elec,
      max(case when s.indicator = 'GDP per capita (current US$)'
              then s.value end) as gdp,
      max(case when s.indicator = 'Current health expenditure (% of GDP)'
              then s.value end) as health_per_capita,
      max(case when s.indicator = 'Energy use (kg of oil equivalent per capita)'
              then s.value end) as energy_use,
      max(case when s.indicator = 'Government expenditure on education, total (% of GDP)'
              then s.value end) as education_expend,
      max(case when s.indicator = 'Individuals using the Internet (% of population)'
              then s.value end) as internet_use,
      max(case when s.indicator = 'Mortality rate, under-5 (per 1,000 live births)'
              then s.value end) as mortality_rate,
      max(case when s.indicator = 'Sci. and tech. journal articles per capita'
              then s.value end) as publication,
      max(case when s.indicator = 'Urban population (% of total population)'
              then s.value end) as urban_population,
      s.income_level_name, s.cluster_label, p.coords
FROM all_country_with_segmentation s 
LEFT JOIN countries_poligons p ON s.country = p.name_en -- добавляю полигоны
WHERE s.cluster_label IS NOT NULL AND p.coords_type != 'all_country_borders_points' -- убираю координаты столиц
group by s.country, s.year, s.income_level_name, s.cluster_label, p.coords;
```
и добавлены индексы по странам и годам:

```sql
CREATE INDEX idx_countries_segment_poligons_country_year ON countries_segments_poligons(country, year);

CREATE INDEX idx_countries_segment_poligons_year ON countries_segments_poligons(year);
```

#### Создание дашборда

В Datalens создал подключение к базе в Supabase, в качестве источника данных взята витрина данных `countries_segments_poligons`.

Сделанный дашборд можно посмотреть **[по ссылке](https://datalens.yandex/0jtlj0h44hyal)**.